In [ ]:
pip install openai

In [ ]:
!pip install datasets

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install transformers evaluate wandb datasets accelerate peft bitsandbytes trl fsspec -U  -qq

    basepath = '/content/drive/MyDrive/Data'

else:
    basepath = '/C:/Users/Vishu/Downloads/Fall_2024/App_NLP'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import openai
import pandas as pd
from datasets import load_dataset
from pathlib import Path

In [ ]:
base_folder = Path(basepath)
data_folder = base_folder/'emotion-detection-bert-fall-2024'
model_folder = base_folder/'Models/Emotion_detection_Llama_languagehead_fall_2024'

In [ ]:
model_folder.mkdir(exist_ok=True, parents = True)

In [ ]:
model_folder

PosixPath('/content/drive/MyDrive/Data/Models/Emotion_detection_Llama_languagehead_fall_2024')

In [ ]:
test_dataset = data_folder/'test.csv'

In [ ]:
from datasets import load_dataset

# Load the test dataset from CSV files
test_dataset = load_dataset('csv', data_files={'test': str(data_folder / 'test.csv')})['test']

In [ ]:
# Define emotion labels
emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

In [ ]:
from google.colab import userdata

In [ ]:
!pip install openai==0.28

In [ ]:
!pip show openai

Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, requests, tqdm
Required-by: 


In [ ]:
import openai

openai.api_key = userdata.get("OPENAI_API_KEY")

try:
    response = openai.Model.list()
    print("API Key is working. Available models:", [model['id'] for model in response['data']])
except Exception as e:
    print("An error occurred:", str(e))

API Key is working. Available models: ['text-embedding-3-small', 'gpt-4o-realtime-preview', 'dall-e-2', 'gpt-4o-realtime-preview-2024-10-01', 'o1-mini-2024-09-12', 'gpt-4-1106-preview', 'o1-mini', 'gpt-3.5-turbo-instruct', 'babbage-002', 'davinci-002', 'whisper-1', 'dall-e-3', 'text-embedding-ada-002', 'gpt-3.5-turbo-16k', 'gpt-4-0125-preview', 'o1-preview-2024-09-12', 'gpt-4-turbo-preview', 'omni-moderation-latest', 'gpt-4o-2024-05-13', 'omni-moderation-2024-09-26', 'tts-1-hd-1106', 'chatgpt-4o-latest', 'gpt-4', 'gpt-4-0613', 'o1-preview', 'gpt-4o', 'gpt-3.5-turbo', 'gpt-4o-2024-08-06', 'gpt-3.5-turbo-0125', 'gpt-4o-mini', 'gpt-4o-mini-2024-07-18', 'gpt-4o-2024-11-20', 'gpt-4-turbo', 'tts-1-hd', 'gpt-4-turbo-2024-04-09', 'gpt-3.5-turbo-1106', 'gpt-4o-audio-preview', 'gpt-4o-audio-preview-2024-10-01', 'tts-1', 'tts-1-1106', 'gpt-3.5-turbo-instruct-0914', 'text-embedding-3-large']


In [ ]:
# Define the prompt template for few-shot classification
few_shot_examples = """
Classify the following tweet into one or more applicable emotions from the list below:
["anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust"]

Example 1:
Tweet: "I can't believe this happened. It's so unfair!"
Emotions: anger, sadness

Example 2:
Tweet: "Looking forward to the big event tomorrow!"
Emotions: anticipation, optimism

Example 3:
Tweet: "This food tastes horrible!"
Emotions: disgust

Now classify this tweet:
Tweet: "{}"

Please respond with a comma-separated list of emotions.
"""

# Function to call ChatGPT for few-shot classification
def classify_tweet_few_shot(tweet):
    try:
        prompt = few_shot_examples.format(tweet)
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Use "gpt-4" if available for better results
            messages=[
                {"role": "system", "content": "You are an expert emotion classifier."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,  # Adjust based on expected response length
            temperature=0.0  # Set to 0 for deterministic outputs
        )
        # Extract and clean the model's output
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"Error classifying tweet: {tweet}\n{e}")
        return ""  # Return an empty string if an error occurs

# Run inference on the test dataset
test_predictions = []
for tweet in test_dataset['Tweet']:
    predicted_emotions = classify_tweet_few_shot(tweet)
    test_predictions.append(predicted_emotions)

# Convert predictions to one-hot encoding
def one_hot_encode(predicted_emotions, class_names):
    # Handle unexpected response formats gracefully
    predicted_emotions = [emotion.strip() for emotion in predicted_emotions.split(",") if emotion.strip() in class_names]
    return [1 if emotion in predicted_emotions else 0 for emotion in class_names]

# Extract IDs and convert predictions to one-hot encoding
test_ids = test_dataset['ID']
binary_predictions = [one_hot_encode(prediction, emotions) for prediction in test_predictions]

# Combine IDs and binary predictions into a DataFrame
df = pd.DataFrame(binary_predictions, columns=emotions)
df.insert(0, "ID", test_ids)  # Add the ID column at the start

# Save the results to a CSV file
output_file = "test_predictions_with_ids_few_shot.csv"
df.to_csv(output_file, index=False)
print(f"ID and one-hot encoded labels saved to '{output_file}'")

# Display the DataFrame
print(df.head())

ID and one-hot encoded labels saved to 'test_predictions_with_ids_few_shot.csv'
           ID  anger  anticipation  disgust  fear  joy  love  optimism  \
0  2018-01559      1             1        0     0    0     0         0   
1  2018-03739      0             0        1     1    0     0         0   
2  2018-00385      1             0        0     0    0     0         0   
3  2018-03001      0             0        0     0    0     0         0   
4  2018-01988      0             0        0     1    0     0         0   

   pessimism  sadness  surprise  trust  
0          0        0         0      0  
1          0        1         0      0  
2          0        1         0      0  
3          0        1         0      0  
4          0        1         0      0  


# New Section